In [1]:
"""
Three cases used by the model: winter (case1), fall turnover (case 2), stratified season (case 3).
For each case, computation of the temperatures, solids concentration and thermocline
depth at the next time step. 

TO FILL!
"""

import numpy as np
import sys
from lake_functions import alphaT,rho_TCss, seasonal_therm
from statistics import mean

def case1(VAR_LAKE,Twinter=4,hmax=8):
    """
    Winter case: ice-covered lake, no heat fluxes = constant temperatures 
    and concentrations. 
     
    INPUTS:
        VAR_LAKE: dictionary containing
            T_epi, T_hypo: epilimnion and hypolimnion temperatures [°C]
            Css_epi, Css_hypo: epilimnion and hypolimnion solids concentrations [mg/L]
            h_epi: thermocline depth [m]
        Twinter: temperature of maximum density [°C]
        hmax: lake depth [m]
        
    OUTPUTS:
        VAR_LAKE: dictionary containing the same variables than VAR_LAKE but at the next time step
    """

    # -------- CODE --------
    # Temperature
    T_epi1 = Twinter
    T_hypo1 = Twinter
    # solid concentrations
    Css_epi1 = VAR_LAKE["Css_epi"]
    Css_hypo1 = VAR_LAKE["Css_hypo"] # considering multiple years, using a variable parameter

    h_epi1 = hmax
    
    VAR_LAKE={"T_epi":T_epi1,"T_hypo":T_hypo1,"Css_epi":Css_epi1,"Css_hypo":Css_hypo1,"h_epi":h_epi1}

    return VAR_LAKE

In [2]:

def case2(VAR_LAKE,Hsurf,Hsw0,tyear,iceon,Vs,C_FFT,hmax=8,g=9.81,Cpw=4200,Twinter=4,A0=7.8*10**6):
    """
    Fall turnover case: one box with heat fluxes.
     
    INPUTS:
        VAR_LAKE: dictionary containing
            T_epi, T_hypo: epilimnion and hypolimnion temperatures [°C]
            Css_epi, Css_hypo: epilimnion and hypolimnion solids concentrations [mg/L]
            h_epi: thermocline depth [m]
        Hsurf: surface heat flux [W/m^2]
        Hsw0: surface radiation flux [W/m^2]
        tyear: time as DOY
        iceon: iceon date as DOY
        Vs: settling velocity [m/s]
        C_FFT: solids concentration in the FFT [g/m^3]
        hmax: lake depth [m]
        g: gravitational acceleration [m/s^2]
        Cpw: heat capacity of water [J.kg^(-1).K^(-1)]
        Twinter: temperature of maximum density [°C]
        A0: lake surface area [m^2]
        
        
    OUTPUTS:
        VAR_LAKE: dictionary containing the same variables than VAR_LAKE but at the next time step
        iceon: iceon date as DOY (modified only if ice formation after initial iceon date)
    """
    # -------- CODE --------
    # ----Initial setup----
    h_epi2 = hmax  # Thermocline depth
    T2 = VAR_LAKE["T_epi"]  # Temperature, constant
    Css2 = VAR_LAKE["Css_epi"] # Initial solid concentration constants
    
    # ----Special process in fall----
    # --Redistribution--
    alpha2 = alphaT(T2)
    kd = 0.325 + 66 * Css_epi2 / 1000  # mg/L
    B = alpha2*g*(Hsurf + Hsw0*(1-np.exp((0.325+66*Css2)*hmax))) / (rho_w0 * Cpw)

    # --stability and concentration calculate--
    if B > 0: # unstable, fully mixed, new epi. depth is constant
        h_epi = hmax
# ABOUT this part, I'm not sure if I understood the comments and process correctly. I just wrote this as a temporary version to see if they work.
        # Hsw0, consider different layer # ALSO, A BIT CONFUSED WITH THIS COMMENT
        Qnet2 = -(Hsurf + Hsw0)*A0
        dT2 = (Qnet2/(m_w0*Cpw))/24*60*60
        Tepi_new = T2 + dT2
        Thypo_new = Tepi_new
        
        if Tepi_new <= 4: #additional safety check
            if t[k] > 200:
                iceon = t[k]
            else:
                print("Warning, Ice in Summer!")
                Tepi_new = Twinter
                Thypo_new = Twinter
        else: # no ice
            # calculate velocities
            rho_30 = rho_TCss(T2, 30)
            N_fft2 = g*(rho_30-rho_w0)/(0.5*rho_fft0) 
            Vr = (1+2*0.2)*B/(N_fft2*hmax)
            # water mass
            m_w0 = rho_w0*A0*hmax 
            # new concentration
            C_sed = -Css2*Vs*A0*24*60*60/A0*hmax
            C_red = C_FFT*Vr*A0*24*60*60/A0*hmax
            Cssnew2 = Css2 + C_sed + C_red
            Csse_new = Cssnew2
            Cssh_new = Cssnew2
            # safety check
            if 
    else:
        # stable
        # calculate thermal structure, stratified
        # image 4
        # linear temperature, so the Tave is Tsur - Thypo/2
        # this time Hepi is really close to the bottom, but if enough time please consider this small stratification
    
    # iceon
    if T_epi <= Twinter:
        iceon = tyear

    VAR_LAKE={"T_epi":Tepi_new,"T_hypo":Thypo_new,"Css_epi":Csse_new,"Css_hypo":Cssh_new,"h_epi":h_epi2}
    return VAR_LAKE, iceon

In [ ]:
def case3(VAR_LAKE,Hsurf,Hsw0,tyear,iceon,Vs,ustar,hmax=8,g=9.81,Cpw=4200,Twinter=4,A0=7.8*10**6,Dth=1.4E-7):
    """
    spring-summer, 2 boxes.
     
    INPUTS:
        VAR_LAKE: dictionary containing
            T_epi, T_hypo: epilimnion and hypolimnion temperatures [°C]
            Css_epi, Css_hypo: epilimnion and hypolimnion solids concentrations [mg/L]
            h_epi: thermocline depth [m]
        Hsurf: surface heat flux [W/m^2]
        Hsw0: surface radiation flux [W/m^2]
        tyear: time as DOY
        iceon: iceon date as DOY
        Vs: settling velocity [m/s]
        ustar: friction velocity [m/s]
        hmax: lake depth [m]
        g: gravitational acceleration [m/s^2]
        Cpw: heat capacity of water [J.kg^(-1).K^(-1)]
        Twinter: temperature of maximum density [°C]
        A0: lake surface area [m^2]
        Dth: thermal molecular diffusivity [m^2/s]
        
  
    OUTPUTS:
        VAR_LAKE: dictionary containing the same variables than VAR_LAKE but at the next time step
        iceon: iceon date as DOY (modified only if ice formation after initial iceon date)
    """
    # --- import necessary equations from other script ---
    from lake_model import Css_epi
    from lake_functions import rho_TCss, seasonal_therm

    # ---- initial setup ---
    T_epi = VAR_LAKE["T_epi"]
    T_hypo = VAR_LAKE["T_hypo"]
    Css_epi = VAR_LAKE["Css_epi"]
    Css_hypo = VAR_LAKE["Css_hypo"]
    h_epi = VAR_LAKE["h_epi"]

    # --- compute densities ---
    rho_epi = rho_TCss(T_epi, Css_epi) # water density [kg/m^3]
    rho_hypo = rho_TCss(T_hypo, Css_hypo)
    rho_avg = 0.5 * (rho_epi + rho_hypo)

    # --- compute N2---
    N2 = g * (rho_hypo - rho_epi) / (0.5 * hmax)

    # --- compute buoyancy flux ---
    alpha = alphaT(T_epi)
    B_flux = alpha * g * (Hsurf + Hsw_epi) / (rho_epi * Cp) if (Hsurf + Hsw_epi) > 0 else 0
# --- constants ---
    Cp = Cpw
    dt = 86400 # time step [s]

    # --- Additional information: shortwave-radiative flux: beer-lambert law ---
    c0 = 0.325
    c1 = 66  # m^-1 * (g/L)^-1
    kd = c0 + c1 * (Css_epi / 1000)  # Css in g/L
    Hsw_epi = Hsw0 * (1 - np.exp(-kd * h_epi))

    # --- Additional information: vertical diffusive heat flux ---
    dTdz = (T_epi - T_hypo) / (0.5 * hmax)
    H_diff = -rho_avg * Cp * (Dth + Kz) * dTdz

    # --- net heat fluxes ---
    H_epi_net = -Hsurf - Hsw_epi + H_diff
    H_hypo_net = - Hsw0 + Hsw_epi - H_diff

    # --- thermocline depth ---
    h_epi_new = seasonal_therm(h_epi, dt, B_flux, N2)
    dh = h_epi_new - h_epi  # change in the thermocline depth
    dhdt = abs(dh / dt) # absolute value because only looks at velocity, not direction

    # --- convective velocity (only if B > 0) ---
    v_therm = dh / dt # velocity of thermocline change
    if B_flux > 0:
        v_conv = 0.6 * (B_flux * h_epi)(1/3) # convective velocity
else:
        v_conv = 0

    # --- settling velocity in epilimnion ---
    vs_epi = max(0, Vs - v_conv - abs(v_therm))
    vs_hypo = Vs

    # --- mass flux from sedimentation ---
    F_sed_epi = dt * vs_epi * A0 * Css_epi # sedimentation from epi to hypo [mg]
    F_sed_hypo = dt * vs_hypo * A0 * Css_hypo

    # --- mass added due to thermocline motion ---
    if dh > 0: # thermocline deepens → hypo water moves to epi
        F_red = Css_hypo * A0 * dh
    else: # thermocline shallows → epi water moves to hypo
        F_red = Css_epi * A0 * abs(dh)

    # --- mass balances ---
    Mss_epi = Css_epi * h_epi * A0 - F_sed_epi + F_red
    Mss_hypo = Css_hypo * (hmax - h_epi) * A0 + F_sed_epi - F_red - F_sed_hypo

    # --- Updated concentrations ---
    Css_epi_new = max(0, Mss_epi / (h_epi_new * A0))
    Css_hypo_new = max(0, Mss_hypo / ((hmax - h_epi_new) * A0))

    # --- updated temperatures ---
    T_epi_new = T_epi + H_epi_net * dt / (rho_epi * Cp * h_epi_new)
    T_hypo_new = T_hypo + H_hypo_net * dt / (rho_hypo * Cp * (hmax - h_epi_new))

    # --- What if after calculation we get an unstable or mixed situation? ---
    if rho_TCss(T_epi_new, Css_epi_new) >= rho_TCss(T_hypo_new, Css_hypo_new) or h_epi_new > (hmax - 0.1): # completely mixed: one bow model
        # switch to 1-box model
        pass
    # --- What if we have some ice signal? ---
    if T_epi_new <= 4:
        if tyear > 200: # re-feezing scenario
        # switch to ice case
        pass

    VAR_LAKE_NEW = {"T_epi": T_epi_new, "T_hypo": T_hypo_new, "Css_epi": Css_epi_new, "Css_hypo": Css_hypo_new, "h_epi": h_epi_new}

    return VAR_LAKE_NEW, iceon, F_red, F_sed_epi, F_sed_hypo